In [2]:
# save model
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()   
drive = GoogleDrive(gauth)
model.save('model.h5')
model_file = drive.CreateFile({'title' : 'model.h5'})                
model_file.SetContentFile('model.h5')                      
model_file.Upload()
drive.CreateFile({'id': model_file.get('id')})

KeyboardInterrupt: ignored

In [0]:
file_obj = drive.CreateFile({'id': '1RXfXjPDVBkJxqUHBmOvyxHfpCj1yw-XO'})                       
file_obj.GetContentFile('model_1.h5')

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

import os
import gzip
import zipfile
import pandas as pd
import numpy as np
import tensorflow as tf
import scipy.io.wavfile as wav
import csv
import librosa
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import IncrementalPCA, PCA
from sklearn.preprocessing import MinMaxScaler

#kreas
import tensorflow.keras as keras
from keras.preprocessing import sequence
from tensorflow.keras import Input, layers
from tensorflow.keras import backend as K
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from tensorflow.keras.callbacks import  History, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
#


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


Using TensorFlow backend.


In [0]:

data = pd.DataFrame(columns=['label', 'feature'])
#data_dir = "/content/gdrive/My Drive/TrainTmp/Train/"
data_dir = "/content/gdrive/My Drive/Handout (1)/Train/Train/"
_len = 0
input_duration = 3
maxx = 0
minn = 1000
with open('/content/gdrive/My Drive/Handout (1)/train_label.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader)    # bỏ qua dòng đầu tiên của file csv
    for row in csv_reader:
      _len += 1
      
      X, sample_rate = librosa.load(data_dir + row[0], res_type='kaiser_fast',duration=None,sr=22050*2,offset=0.0)
      #sample_rate, X = wav.read(data_dir + row[0])
      #sample_rate = np.array(sample_rate)
      mfccs = np.mean(librosa.feature.mfcc(y=X, sr=22050, n_mfcc=20), axis=0)
      feature = mfccs
      data.loc[_len, 'feature'] = feature
      data.loc[_len, 'label'] = int(row[1])
      maxx  = max(maxx, feature.shape[0])
      minn  = min(minn, feature.shape[0])
      print(_len)
      
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153


In [0]:
#def extract_feature(X, sample_rate): 
#    stft = np.abs(librosa.stft(X))
#    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
   
#    return mfccs



In [0]:
#test DNN
#X = pd.DataFrame(data['feature'].values.tolist())
#X = np.array(data['feature'])
#Y = np.array(data['label'])
#X = np.array(X)
#features = np.empty((0,193))
#for i in range(0, _len):
  
#  mfccs= extract_feature(X[i ], sample_rate[i])
#  X[i] = mfccs
#  print(i)


In [0]:

for i in range(1, _len + 1):
  Tmp = np.zeros(maxx)
  dat  = data['feature'].loc[i]

  for j in range(dat.shape[0]):
    Tmp[j] = dat[j]
  data['feature'].loc[i] = Tmp


In [0]:
X = pd.DataFrame(data['feature'].values.tolist())
#X = np.array(data['feature'])
#Y = np.array(data['label'])
X = np.array(X)
Y = pd.DataFrame(data['label'].values.tolist())
Y = np.array(Y)
Y = np.resize(Y, _len)
X = np.resize(X, (_len, maxx))

In [26]:
X.shape

(10, 83)

In [0]:
#PCA (Thu nghiem khong thanh cong)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=100).fit(X_scaled)
X_pca = pca.transform(X_scaled)



(5229, 100)

In [0]:
def plot_time_series(data):
    """
    Plot the Audio Frequency.
    """
    fig = plt.figure(figsize=(14, 8))
    plt.title('Raw wave ')
    plt.ylabel('Amplitude')
    plt.plot(np.linspace(0, 1, len(data)), data)
    plt.show()


def noise(data):
    """
    Adding White Noise.
    """
    # you can take any distribution from https://docs.scipy.org/doc/numpy-1.13.0/reference/routines.random.html
    noise_amp = 0.005*np.random.uniform()*np.amax(data)
    data = data.astype('float64') + noise_amp * np.random.normal(size=data.shape[0])
    return data
    
def shift(data):
    """
    Random Shifting.
    """
    s_range = int(np.random.uniform(low=-5, high = 5)*500)
    return np.roll(data, s_range)
    
def stretch(data, rate=0.8):
    """
    Streching the Sound.
    """
    data = librosa.effects.time_stretch(data, rate)
    return data
    
def pitch(data, sample_rate):
    """
    Pitch Tuning.
    """
    bins_per_octave = 12
    pitch_pm = 2
    pitch_change =  pitch_pm * 2*(np.random.uniform())   
    data = librosa.effects.pitch_shift(data.astype('float64'), 
                                      sample_rate, n_steps=pitch_change, 
                                      bins_per_octave=bins_per_octave)
    return data
    
def dyn_change(data):
    """
    Random Value Change.
    """
    dyn_change = np.random.uniform(low=1.5,high=3)
    return (data * dyn_change)
    
def speedNpitch(data):
    """
    peed and Pitch Tuning.
    """
    # you can change low and high here
    length_change = np.random.uniform(low=0.8, high = 1)
    speed_fac = 1.0  / length_change
    tmp = np.interp(np.arange(0,len(data),speed_fac),np.arange(0,len(data)),data)
    minlen = min(data.shape[0], tmp.shape[0])
    data *= 0
    data[0:minlen] = tmp[0:minlen]
    return data


(5229, 65)

In [0]:
#X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, random_state = 42, shuffle = True)

#clf = SVC(kernel = 'rbf', probability=True)


#clf.fit(X_train, y_train)

#print(accuracy_score(clf.predict(X_val), y_val))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.27724665391969405


In [0]:

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, random_state = 42, shuffle = True)


In [0]:
#tach tap cho PCA
X_train, X_val, y_train, y_val = train_test_split(X_pca, Y, test_size = 0.2, random_state = 42, shuffle = True)





---

# CNN




In [0]:
X_train

array([[-21.20674559, -20.00929564, -19.51511234, ...,   0.        ,
          0.        ,   0.        ],
       [-17.51477023, -17.5939196 , -17.7561133 , ...,   0.        ,
          0.        ,   0.        ],
       [-18.94902082, -18.57924443, -18.37237334, ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [-16.75036715, -17.39210631, -19.75672577, ...,   0.        ,
          0.        ,   0.        ],
       [-19.44917117, -17.88207168, -18.34873679, ...,   0.        ,
          0.        ,   0.        ],
       [-17.3022671 , -17.31017311, -16.53847264, ...,   0.        ,
          0.        ,   0.        ]])

In [0]:
X_train.shape

(4183, 157)

In [0]:
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
lb = LabelEncoder()
y_traincnn = np_utils.to_categorical(lb.fit_transform(y_train))
y_valcnn = np_utils.to_categorical(lb.fit_transform(y_val))



X_traincnn = np.expand_dims(X_train, axis=2)
X_valcnn = np.expand_dims(X_val, axis=2)


In [0]:

num_fc = 64
batch_size = 32
num_epochs = 1500 #best model will be saved before number of epochs reach this value
learning_rate = 0.0001
decay = 1e-6
momentum = 0.9

In [0]:
X_train.shape

(4183, 157)

(5229, 157)

In [0]:
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def fscore(y_true, y_pred):
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    f_score = 2 * (p * r) / (p + r + K.epsilon())
    return f_score

def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr

LTSM + CNN








In [0]:
#@title Default title text
#LSTM + CNN 
model = Sequential(name='Emo1DD')
	
# LFLB1
model.add(Conv1D(filters = 64,kernel_size = (3),strides=1,padding='same',data_format='channels_last',input_shape=(X_train.shape[1],1)))	
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling1D(pool_size = 4, strides = 4))

	#LFLB2
model.add(Conv1D(filters=64, kernel_size = 3, strides=1,padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling1D(pool_size = 4, strides = 4))

	#LFLB3
model.add(Conv1D(filters=128, kernel_size = 3, strides=1,padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling1D(pool_size = 4, strides = 4))
	#LFLB4
model.add(Conv1D(filters=128, kernel_size = 3, strides=1,padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling1D(pool_size = 4, strides = 4))

	#LSTM
model.add(LSTM(units=num_fc)) 
		
	#FC
model.add(Dense(units=6,activation='softmax'))

	#Model compilation	
opt = tf.keras.optimizers.SGD(lr = learning_rate, decay=decay, momentum=momentum, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', fscore])

CNN


In [0]:

#CNN
model = Sequential()
model.add(Conv1D(256, 8, padding='same',input_shape=(X_train.shape[1],1)))
model.add(Activation('relu'))
model.add(Conv1D(256, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(6))
model.add(Activation('softmax'))
opt = tf.keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay= 1e-6, nesterov=False)
#opt = tf.keras.optimizers.SGD(lr=1e-3, momentum=0.3, decay=0, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', fscore])

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_47 (Conv1D)           (None, 157, 256)          2304      
_________________________________________________________________
activation_44 (Activation)   (None, 157, 256)          0         
_________________________________________________________________
conv1d_48 (Conv1D)           (None, 157, 256)          524544    
_________________________________________________________________
batch_normalization_44 (Batc (None, 157, 256)          1024      
_________________________________________________________________
activation_45 (Activation)   (None, 157, 256)          0         
_________________________________________________________________
dropout (Dropout)            (None, 157, 256)          0         
_________________________________________________________________
max_pooling1d_44 (MaxPooling (None, 19, 256)           0

In [21]:
#mo hinh train chung cho cac model

es = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=20, min_lr=0.000001)
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=8)
#mc = ModelCheckpoint('best_model.h5', monitor='val_categorical_accuracy', mode='max', verbose=1, save_best_only=True)
mc = ModelCheckpoint('model_1.h5', save_best_only=True, monitor='val_loss', mode='min')
history = model.fit(X_traincnn, y_traincnn,epochs=num_epochs,batch_size=batch_size,validation_data=(X_valcnn,y_valcnn),callbacks=[es, mc])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 400 samples, validate on 100 samples
Epoch 1/1500
 96/400 [======>.......................] - ETA: 1:03 - loss: 1.7905 - acc: 0.1562 - fscore: 0.0000e+00

KeyboardInterrupt: ignored

# **SVM**

In [0]:
# Define the paramter grid for C from 0.001 to 10, gamma from 0.001 to 10
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, random_state = 42, shuffle = True)

C_grid = [0.001, 0.01, 0.1, 1, 10]
gamma_grid = [0.001, 0.01, 0.1, 1, 10]
param_grid = {'C': C_grid, 'gamma' : gamma_grid}

grid = GridSearchCV(SVC(kernel='rbf'), param_grid, cv = 3, scoring = "accuracy")
grid.fit(X_train, y_train)

# Find the best model
print(grid.best_score_)

print(grid.best_params_)

print(grid.best_estimator_)

0.3452067893856084
{'C': 1, 'gamma': 0.001}
SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [0]:
clf = SVC(kernel = 'rbf',C=1,gamma=0.001,decision_function_shape='ovr')

clf.fit(X_train, y_train)

print(accuracy_score(clf.predict(X_val), y_val))

0.35564053537284895


In [0]:
label = clf.predict(X_val)

In [0]:
count = 0
for i in range(label.shape[0]):
  if (label[i] == y_val[i]):
    count +=1;
print(count)

851


In [0]:
label.shape[0]

1046

In [4]:
%cd '/content/gdrive/My Drive/Handout (1)'
! ls

/content/gdrive/My Drive/Handout (1)
aug_noiseNshift_2class2_np.h5  model_1.h5  model.h5  Train  train_label.csv


In [0]:
! pip install os


ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [3]:
!pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5889 sha256=a76f778309591cf07a95a8eef7a5b7dface865580d2be26dede6e21165167049
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features
